In [88]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [145]:
nx, ny = 20, 20   # dimensions of the reservior space
dx, dy = 100, 100   # the distance (in ft.) each cell in the grid block covers
dt = .001            # This is the timestep in days that the model will iterate through
nt = 1000000          # This is the total number of steps I want to simulate in this simple reservoir model

# Reservoir properties

k = 100  # permeability of the reservoir in (md)
mu = 1.0    # Viscosity (cp)
phi = .1    # porosity
ct = 1e-6   # compressibility (psi^-1)

# Conversion factor for field units
alpha = (0.0002637 * k) / (phi * mu * ct)  # diffusivity in ft^2/day

In [ ]:
# Initialize the Reservoir Pressure Grid
P_init = 4000     #Initial pressure in (psi)
P = np.ones((ny, nx)) * P_init

# Add a pressure sink in the middle of the grid
P[ny // 2, nx // 2] = 1000

#Step through the time steps and simulate the flow through the reservoir grid
for n in range(nt):
    P_new = P.copy()
    for i in range(1, ny - 1):      # rows (y-direction)
        for j in range(1, nx - 1):  # columns (x-direction)
            d2P_dx2 = (P[i, j+1] - 2*P[i, j] + P[i, j-1]) / dx**2
            d2P_dy2 = (P[i+1, j] - 2*P[i, j] + P[i-1, j]) / dy**2
            P_new[i, j] += alpha * dt * (d2P_dx2 + d2P_dy2)
            
    #Establish the boundary conditions to keep them fixed
    P = P_new.copy()
    P[0, :] = P_init
    P[-1, :] = P_init
    P[:, 0] = P_init
    P[:, -1] = P_init

In [ ]:
# Helper function to plot pressure
def plot_pressure(P, t):
    plt.figure(figsize=(6, 5))
    plt.imshow(P, cmap='viridis', origin='lower')
    plt.colorbar(label='Pressure (psi)')
    plt.title(f"Pressure Distribution at Time = {t} days")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.tight_layout()
    plt.show()

In [ ]:
plot_pressure(P, nt*dt)

In [ ]:
P